<H3> AUTOMATING INGESTION FOR AZURE DATA STUDIO(SQL SERVER) </H3>

<H5> Libraries </H5>

In [2]:
import pyodbc 
import pandas as pd
import os
import shutil
import re

<h6>Establish Connection</h6>

In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=localhost\SQLEXPRESS08;'
                          'Database=master;'
                          'Trusted_Connection=yes;',autocommit=True)
print("done")
cursor = conn.cursor()
print("cursor established!")

done
cursor established!


<H6> Creating Variable to Store Path & Database Name</H6>

In [4]:
dir_path = os.getcwd() ##change directory to the directory that contains all excel files
os.chdir(dir_path) 
path = os.getcwd() ##storing the path of current directory as variable

In [5]:
folder_nm = "testing" ##used to create new folder 
new_db_nm = "new_database" ##database name
os.system('mkdir {}'.format(folder_nm)) #
new_path=path+"\\"+folder_nm 

<h6> Dictionary used to automate Create table Statement </h6>

In [6]:
replacements = {
        'timedelta64[ns]': 'nvarchar(255)',
        'object': 'nvarchar(255)',
        'float64': 'float',
        'int64': 'int',
        'bool': 'string',
        'datetime64': 'timestamp',
        'datetime64[ns]':'nvarchar(255)'
    }

<H3> FUNCTIONS </H3>

<H6> Function to filter only file names </h6>

In [7]:
def list_csvs_excels(path):
    list_fil = os.listdir(path) ##list of files in the current directory
    list_csv_xlsx = [i for i in list_fil if i.endswith((".csv",".xlsx"))] ##filtering names of only excel & csv files 
    return list_csv_xlsx

<H6> Moving List of csvs </h6>

In [8]:
def moving_files(list_csvs,path,folder_nm):
    for i in list_csvs:
        source_path = path.replace(os.sep, '/')+'/'+ i
        destination_path  = path.replace(os.sep, '/')+'/'+ folder_nm+ '/'+i
        shutil.copy(source_path,destination_path)
    print("completed")

<h6> Function to read csv & excel files </h6>

In [9]:
def read_csv(file,newpath):
    df = pd.read_csv((newpath+"\\" +file))
    return df

def read_ex(file,newpath):
    fil = (newpath+"\\" +file)
    df = pd.read_excel(fil) #encoding="ISO-8859-1"
    return df

##this function will read  file depending on file extension
def main_func(file,newpath):
    ops = {"csv":read_csv,"xlsx":read_ex,"xls":read_ex}
    chose_opern = ops.get(file.split(".")[-1])
    return chose_opern(file,newpath)

<h6> Function to create database conn & create database </h6>

In [10]:
def db_conn(new_db_nm):
    cursor.execute("""IF NOT EXISTS 
           (
         SELECT name FROM dbo.sysdatabases 
         WHERE name = N'%s'
            )
         CREATE DATABASE [%s]"""%(new_db_nm,new_db_nm))
    
    print("db if not existed created")
    
    cursor.execute("""USE %s;""" % (new_db_nm))
    print("use db: %s" %(new_db_nm))

<h6> Creating Table</h6>

In [11]:
def create_tbl(new_db_nm,table_nm,tbl_stat):
    cursor.execute("Use %s;" % new_db_nm)
    cursor.execute("DROP TABLE IF EXISTS %s;" %table_nm)
    cursor.execute("create table %s (%s);" % (table_nm, tbl_stat))
    print("Table %s created" %table_nm)

<h4> Applying Function to Automate Process </h4>

In [12]:
# conn_established()
list_csv_xls = list_csvs_excels(path)[40:45] #fileting list of csvs
list_csv_xls
moving_files(list_csv_xls,path,folder_nm)
db_conn(new_db_nm)
list_csv_xls

completed
db if not existed created
use db: new_database


['matches_kkr and rcb.csv',
 'matches_SHORT.csv',
 'matches_SHORT.xlsx',
 'men_t20i_team_batting_stats.csv',
 'men_t20i_team_bowling_stats.csv']

In [13]:
for i in list_csv_xls: 
    try:
         try :
            df = main_func(i,new_path)
            df1 = df.fillna(" ")
            del df
            tbl_nm = i.split(".",1)[0].replace(".","_").replace("$","_").replace("&","_").replace("&","_").replace("-","_").replace(" ","").replace("+","_").replace("%","_").replace(r"/","_").replace("\\","_").replace("(","").replace(")","").replace("#","_").replace(";","_").replace(":","_").replace(",","_").replace("'","_").replace('\n',"_").replace("'","_")
            #tbl_nm = re.sub(r'\d', '_', tbl_nm) ##any number treatment 
            columns = [i.replace(".","_").replace("$","_").replace("-","_").replace(" ","").replace("&","_").replace("+","_").replace("%","_").replace(r"/","_").replace("\\","_").replace("(","").replace(")","").replace(" ","").replace("#","_").replace(";","_").replace(":","_").replace(",","_").replace("'","_").replace("'","_").replace('\n',"_") for i in df1.columns]
            df1 = df1.rename(columns = {x:y for x, y in zip(df1.columns, columns)})
            m = df1.dtypes.reset_index().iloc[:,1].astype('str')
            tbl_stat = "  ,".join((" {}  {}").format(o,p) for o,p in list(zip(df1.columns, [replacements[i] for i in m])))#cls
            create_tbl(new_db_nm,tbl_nm,tbl_stat)
            insert_Stat1 = "INSERT INTO {}".format(tbl_nm)+("("+",".join(["["+i+"]"  for i in df1.columns])+")")+ " VALUES " + ("("+",".join(["?" for i in range(0,len(df1.columns))])+")")
            
            for ind,row in df1.iterrows():
                cursor.execute(insert_Stat1,tuple(row))
                conn.commit()
        
            print("Uploaded Table {}"%(tbl_nm))
        
         except:
            df = main_func(i,new_path)
            df1 = df.fillna(" ")
            df1 = df1.astype("str")
            del df
            tbl_nm = i.split(".",1)[0].replace(".","_").replace("$","_").replace("&","_").replace("-","_").replace(" ","").replace("+","_").replace("%","_").replace(r"/","_").replace("\\","_").replace("(","").replace(")","").replace("#","_").replace(";","_").replace(":","_").replace(",","_").replace("'","_").replace('\n',"_").replace("'","_")
            #tbl_nm = re.sub(r'\d', '_', tbl_nm)
            columns = [i.replace(".","_").replace("$","_").replace("-","_").replace(" ","").replace("&","_") .replace("+","_").replace("%","_").replace(r"/","_").replace("\\","_").replace("(","").replace(")","").replace(" ","").replace("#","_").replace(";","_").replace(":","_").replace(",","_").replace("'","_").replace("'","_").replace('\n',"_") for i in df1.columns]
            df1 = df1.rename(columns = {x:y for x, y in zip(df1.columns, columns)})
            m = df1.dtypes.reset_index().iloc[:,1].astype('str')
            tbl_stat = "  ,".join((" {}  {}").format(o,p) for o,p in list(zip(df1.columns, [replacements[i] for i in m])))#cls
            create_tbl(new_db_nm,tbl_nm,tbl_stat)
            insert_Stat1 = "INSERT INTO {}".format(tbl_nm)+("("+",".join(["["+i+"]"  for i in df1.columns])+")")+ " VALUES " + ("("+",".join(["?" for i in range(0,len(df1.columns))])+")")
            
            for ind,row in df1.iterrows():
                cursor.execute(insert_Stat1,tuple(row))
                conn.commit()
            
            print("Uploaded Table {}"%(tbl_nm))      
    except:
        pass
        
print("completed!")      

Table matches_kkrandrcb created
Table matches_kkrandrcb created
Table matches_SHORT created
Table matches_SHORT created
Table matches_SHORT created
Table matches_SHORT created
Table men_t20i_team_batting_stats created
Table men_t20i_team_batting_stats created
Table men_t20i_team_bowling_stats created
Table men_t20i_team_bowling_stats created
completed!
